In [1]:
import yaml

with open('prompt_placeholders.yaml', 'r') as file:
    prompt_placeholders = yaml.safe_load(file)
    with open('project_letzte_wohnung(5).md', 'r') as md_file:
        product_description = md_file.read()
    
    


## First (smart) Select the Lead Followings

In [2]:
from enriching_leads import score_leads

df = score_leads(
    target_desc=prompt_placeholders['TARGET_AUDIENCE_FOLLOWING_DESCRIPTION'],
    target_examples=prompt_placeholders['TARGET_AUDIENCE_FOLLOWING_EXAMPLES'],
    use_perplexity=False,
    csv_in="leads.csv",
    csv_out="leads_scored.csv",
    max_handles=10000,
    min_filter_score=3
)

print(df[['username','score']].head())


Processing leads: 100%|██████████| 1743/1743 [15:36<00:00,  1.86it/s]  

               username  score
0       teamneunzehn.at    4.0
1     patriziamilani_me    3.0
2   obermair_immobilien    5.0
3           krreal.gmbh    5.0
4  anderlage_apartments    4.0


In [7]:
import json
usernames = df["username"].to_list()
chunk_size = 50

for i in range(0, len(usernames), chunk_size):
    chunk = usernames[i:i+chunk_size]
    with open(f'usernames_{i//chunk_size + 1}.json', 'w') as f:
        json.dump(chunk, f)

# Read Scraped Followers File

In [1]:
import pandas as pd
df = pd.read_csv("dataset_instagram-followers-and-following-scraper_2025-05-21_23-08-05-378.csv")
df = df[["username", "user_id", "source_username", "is_private", "is_verified"]]
len(df)

14239

In [5]:
df_grouped = df.groupby("username").agg({
    "is_private": "first",
    "user_id": "first",
    "is_verified": "first",
    "source_username": list
}).reset_index()
df_grouped["count"] = df.groupby("username").size().values
df_grouped.sort_values("count", ascending=False, inplace=True)
print("Len df_grouped", len(df_grouped))
print("Number of private accounts", df_grouped[df_grouped["is_private"] == True].shape[0])
df_grouped["channelName"] = df_grouped["username"]
df_grouped = df_grouped[df_grouped["is_private"] == False] # Can only scrape public accounts
df_grouped.to_csv("dataset_instagram-followers-and-following-scraper_2025-05-21_23-08-05-378_grouped.csv", index=False)
df_grouped


Len df_grouped 12926
Number of private accounts 4538


,username,is_private,user_id,is_verified,source_username,count,channelName
5300,immobilienfotograf.kaernten,False,68968012335,False,"[krreal.gmbh, austriasothebysrealty, natureali...",26,immobilienfotograf.kaernten
5319,immocapture_carinthia,False,69775913679,False,"[teamneunzehn.at, krreal.gmbh, naturealimmobil...",17,immocapture_carinthia
3026,die_immorei,False,70204760635,False,"[teamneunzehn.at, tim_heissenberger, teamwohnw...",12,die_immorei
8724,nh_mat,False,57041265232,False,"[teamneunzehn.at, tim_heissenberger, teamwohnw...",11,nh_mat
7328,malerbetrieb_m4,False,56260036918,False,"[teamwohnwerk, mundwerk.immobilien, wohnreich....",9,malerbetrieb_m4
...,...,...,...,...,...,...,...
4407,gerhardheintschel,False,56612403688,False,[schantl_ith_immobilientreuhand],1,gerhardheintschel
4408,gerhardniesslein,False,46380625629,False,[downhilllodge],1,gerhardniesslein
4409,gerhardsteigenberger,False,24501769998,False,[downhilllodge],1,gerhardsteigenberger
4410,gerichtliche,False,49950647335,False,[austriasothebysrealty],1,gerichtliche


# Enrich Actual Targets

In [3]:
from enriching_leads import score_leads

df = score_leads(
    target_desc=prompt_placeholders['TARGET_AUDIENCE_DESCRIPTION'],
    target_examples=prompt_placeholders['TARGET_AUDIENCE_EXAMPLES'],
    product_desc=product_description,
    use_perplexity=True,
    csv_in="dataset_instagram-followers-and-following-scraper_2025-05-21_23-08-05-378_grouped.csv",
    csv_out="dataset_instagram-followers-and-following-scraper_2025-05-21_23-08-05-378_grouped_scored4.csv",
    max_handles=1000,
    min_filter_score=1
)

print(df[['username','score']].head())


Processing leads:  81%|████████  | 807/1000 [30:29<07:17,  2.27s/it]


                      username  score
0                       nh_mat      1
1               immo.assistant      1
2    innenausbau_j.luszczewski      1
3                   simon.frwt      1
4  immobilienfotograf.kaernten      1


In [30]:
df

,username,score,reasoning,enrichment
0,immobilienfotograf.kaernten,3,"The Instagram account ""immobilienfotograf.kaer...","```json\n[\n {\n ""site"": ""Instagram"",\n ..."
13,kalandra_immobilien,4,"The Instagram account ""kalandra_immobilien"" ap...","```json\n[\n {\n ""site"": ""website"",\n ""..."
20,lvr_gmbh,4,"The Instagram account belongs to LVR GMBH, a c...","```json\n[\n {\n ""site"": ""Website"",\n ""..."
21,teamwohnwerk,5,The Instagram account appears to be focused on...,"```json\n[\n {\n ""site"": ""TikTok"",\n ""u..."
29,7doerfer,5,"The Instagram account ""Sieben Dörfer Immobilie...","```json\n[\n {\n ""site"": ""instagram"",\n ..."
30,pp_immoconsulting,3,The Instagram account belongs to a real estate...,"```json\n[\n {\n ""site"": ""linkedin"",\n ..."
31,papstimmobilien,3,"The Instagram account ""Papst Immobilien"" prima...","```json\n[\n {\n ""site"": ""Immobilienscout2..."
39,rechtsanwalt_orsini_rosenberg,3,The Instagram account appears to belong to a l...,"[\n {\n ""site"": ""website"",\n ""url"": ""ht..."
40,fortuna.real.makler,3,"The Instagram account ""fortuna.real.makler"" ap...","```json\n[\n {\n ""site"": ""linkedin"",\n ..."
42,immo__circle,5,"The Instagram account ""immo__circle"" appears t...","```json\n[\n {\n ""site"": ""Website"",\n ""..."


# Apollo Export

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("apollo-contacts-export.csv")
print(df.columns)

Index(['First Name', 'Last Name', 'Title', 'Company',
       'Company Name for Emails', 'Email', 'Email Status',
       'Primary Email Source', 'Email Confidence',
       'Primary Email Catch-all Status', 'Primary Email Last Verified At',
       'Seniority', 'Departments', 'Contact Owner', 'Work Direct Phone',
       'Home Phone', 'Mobile Phone', 'Corporate Phone', 'Other Phone', 'Stage',
       'Lists', 'Last Contacted', 'Account Owner', '# Employees', 'Industry',
       'Keywords', 'Person Linkedin Url', 'Website', 'Company Linkedin Url',
       'Facebook Url', 'Twitter Url', 'City', 'State', 'Country',
       'Company Address', 'Company City', 'Company State', 'Company Country',
       'Company Phone', 'SEO Description', 'Technologies', 'Annual Revenue',
       'Total Funding', 'Latest Funding', 'Latest Funding Amount',
       'Last Raised At', 'Subsidiary of', 'Email Sent', 'Email Open',
       'Email Bounced', 'Replied', 'Demoed', 'Number of Retail Locations',
       'Apollo Conta

In [8]:
person_columns = [
    'First Name', 'Last Name', 'Title', 'Company', 'Company Name for Emails', 'Email',
    'Work Direct Phone', 'Home Phone', 'Mobile Phone', 'Corporate Phone', 'Other Phone',
    'Seniority', 'Departments', 'Person Linkedin Url', 'Facebook Url', 'Twitter Url',
    'City', 'State', 'Country', 'Company Address', 'Company City', 'Company State',
    'Company Country', 'Company Phone', '# Employees', 'Industry', 'Website',
    'Company Linkedin Url', 'SEO Description', 'Technologies', 'Annual Revenue',
    'Total Funding', 'Latest Funding', 'Latest Funding Amount', 'Last Raised At',
    'Subsidiary of', 'Number of Retail Locations', 'Keywords',
    'Secondary Email', 'Secondary Email Source', 'Tertiary Email', 'Tertiary Email Source'
]
df = df[person_columns]

In [9]:
df.head()

,First Name,Last Name,Title,Company,Company Name for Emails,Email,Work Direct Phone,Home Phone,Mobile Phone,Corporate Phone,...,Latest Funding,Latest Funding Amount,Last Raised At,Subsidiary of,Number of Retail Locations,Keywords,Secondary Email,Secondary Email Source,Tertiary Email,Tertiary Email Source
0,Christoph,Dorner,Leiter HR-Entwicklung,TGA Dorner GmbH,TGA Dorner,NaN,NaN,NaN,NaN,'+43 664 88699192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Johannes,Berger,Sales Manager,Haas Fertigbau Österreich,Haas Fertigbau Österreich,johannes.berger@haas-fertigbau.at,NaN,NaN,NaN,'+43 338 56660,...,NaN,NaN,NaN,NaN,NaN,"baugewerbe, fertighaus, bungalow, fertigteilha...",NaN,NaN,NaN,NaN
2,Katrin,Peinsipp,Event-Manager,ANDY WOLF EYEWEAR,ANDY WOLF EYEWEAR,katrin@andy-wolf.com,NaN,NaN,NaN,'+43 333 261955,...,NaN,NaN,NaN,NaN,NaN,"retail apparel & fashion, eyewear, frames, sun...",NaN,NaN,NaN,NaN
3,Martin,NaN,Sales Manager,Siemens Energy,Siemens Energy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"transmission, industrial applications, renewab...",NaN,NaN,NaN,NaN
4,Petra,Schafzahl,Product Manager,Retter Reisen,Retter Reisen,NaN,NaN,NaN,NaN,'+43 333 53900,...,NaN,NaN,NaN,NaN,NaN,"busreisen, wanderreisen, radreisen, kulturreis...",NaN,NaN,NaN,NaN


In [ ]:
def row_to_string(row):
    return "\n".join(
        f"{col}: {val}"
        for col, val in row.items()
        if pd.notnull(val) and str(val).strip() != ""
    )

info_strings = df.apply(row_to_string, axis=1)
# Example: print the first info string
print(info_strings.iloc[2])

First Name: Katrin
Last Name: Peinsipp
Title: Event-Manager
Company: ANDY WOLF EYEWEAR
Company Name for Emails: ANDY WOLF EYEWEAR
Email: katrin@andy-wolf.com
Corporate Phone: '+43 333 261955
Seniority: Manager
Departments: Marketing
Person Linkedin Url: http://www.linkedin.com/in/katrin-peinsipp-958227320
Facebook Url: https://facebook.com/andywolfeyewear
Twitter Url: https://twitter.com/AndyWolfFashion
City: Fuerstenfeld
State: Styria
Country: Austria
Company Address: 21 Am Okopark, Hartberg, Styria, Austria, 8230
Company City: Hartberg
Company State: Styria
Company Country: Austria
Company Phone: '+43 333 261955
# Employees: 34
Industry: apparel & fashion
Website: https://andy-wolf.com
Company Linkedin Url: http://www.linkedin.com/company/andy-wolf-eyewear
Technologies: MailJet, Microsoft Office 365, Hubspot, Nginx, Mobile Friendly, Google Font API, WordPress.org, Google Analytics, Google Tag Manager, Woo Commerce, Google Maps, 
Annual Revenue: 5236000.0
Keywords: retail apparel & fa

In [22]:
email_prompt="""### Deine Rolle
Du bist ein österreichischer Immobilienentwickler mit genau **einer** fertig ausgestatteten Gartenwohnung im Neubau „Haus 3 – St. Johann i. d. Haide 247“.  
Du möchtest lokale Unternehmen anschreiben und ihnen die Wohnung als ideales Zuhause oder Investment für neue Mitarbeitende vorstellen.  
Schreibe kurze, freundliche, aber professionelle E‑Mails auf Deutsch, die hilfreich klingen und **nicht** stalker‑haft wirken.

### 1 · Kontakt‑ & Firmeninfos  
Verwende folgende Daten als Fakten (nicht alles direkt zitieren!):  
{{ContactInfo}}

### 2 · Recherche‑Notizen  
Nimm Anknüpfungspunkte aus diesen unstrukturierten Erkenntnissen auf:  
{{ResearchNotes}}

### 3 · Projekt‑Daten  
Wähle max. **2** Kernaussagen aus diesen Projektdetails:  
{{ProjectData}}

### E‑Mail‑Anforderungen
- **Anrede:** T‑Form, respektvoll („Liebe Frau/Herr  [Name]“).  
- **Hook:** Kurze aber deutliche Erwähnung eines Research‑Aspekts (z. B. Nachhaltigkeit, Standortnähe).  
- **Kernaussage:** Max. 2 Argumente aus Abschnitt 3, warum die Wohnung ideal für eine*n neue oder bestehende [Company Name]‑Mitarbeiter*in oder als Firmen‑Investment ist.  
- **Verknappung:** Nur **eine** Einheit verfügbar.  
- **Call‑to‑Action:** Flyer/Rundgang als Anhang/Link, bitte um Rückmeldung.  
- **Sprache & Stil:** Deutsch, unter 300 Wörter im Body (ohne Signatur), freundlich aber professionell.  
- **Signatur:** Name, Firma, Telefon, E‑Mail.

### Ausgabeformat  
Gib **nur** den finalen E‑Mail‑Text zurück (Betreff + Body inkl. Signatur). Keine weiteren Erklärungen.

---  
Jetzt: Erstelle die E‑Mail.
Der Absender ist "Alfred Krauck im Namen von Krauck Immobilien"
Bitte generiere keinen Betreff."""

In [20]:
# Extensive research on info_strings.iloc[2] using Perplexity
from utils.utils_llm import query_perplexity, gpt_chat

# Prepare the research prompt
research_prompt = f"Bitte mache eine ausführliche Web-Recherche zu folgender Person/Firma und gib alle relevanten Erkenntnisse strukturiert zurück.\n\n{info_strings.iloc[2]}. Also find out how close the company is to St. Johann i. d. Haide 247"

perplexity_result = query_perplexity(research_prompt)
print(perplexity_result)

Katrin Peinsipp ist Event-Managerin bei ANDY WOLF EYEWEAR, einem Unternehmen aus der Modebranche mit Fokus auf hochwertige, handgefertigte Brillen und Sonnenbrillen. Hier sind die relevanten Erkenntnisse zu ihr und ihrem Arbeitgeber strukturiert zusammengefasst:

## Person Katrin Peinsipp

- **Position:** Event-Managerin  
- **Abteilung:** Marketing  
- **Seniority:** Manager  
- **Kontakt:**  
  - E-Mail: katrin@andy-wolf.com  
  - Telefon: +43 333 261955  
- **LinkedIn:** [Katrin Peinsipp](http://www.linkedin.com/in/katrin-peinsipp-958227320)  
- **Standort:** Fürstenfeld, Steiermark, Österreich  

## Unternehmen ANDY WOLF EYEWEAR

- **Branche:** Apparel & Fashion, spezialisiert auf Brillenmode  
- **Mitarbeiteranzahl:** 34  
- **Jahresumsatz:** ca. 5,236,000 EUR  
- **Firmensitz:**  
  - Adresse: 21 Am Okopark, Hartberg, Steiermark, Österreich, 8230  
  - Telefon: +43 333 261955  
- **Webseite:** https://andy-wolf.com  
- **LinkedIn:** [ANDY WOLF EYEWEAR](http://www.linkedin.com/com

In [23]:
# Generate a custom email using the research result and email_prompt with ChatGPT
# Fill the placeholders in email_prompt
contact_info = info_strings.iloc[2]
research_notes = perplexity_result
project_data = product_description if 'product_description' in locals() else ""

filled_prompt = email_prompt.replace("{{ContactInfo}}", contact_info)
filled_prompt = filled_prompt.replace("{{ResearchNotes}}", research_notes)
filled_prompt = filled_prompt.replace("{{ProjectData}}", project_data)

messages = [
    {"role": "system", "content": "Du bist ein professioneller Immobilienverkäufer."},
    {"role": "user", "content": filled_prompt}
]

email_text = gpt_chat(messages, model="gpt-4o", temperature=0.4, max_tokens=512)
print(email_text)

Liebe Frau Peinsipp,

ich hoffe, es geht Ihnen gut. Ich bin auf ANDY WOLF EYEWEAR und Ihr Engagement für Nachhaltigkeit aufmerksam geworden, was mich sehr beeindruckt hat. In diesem Zusammenhang möchte ich Ihnen eine interessante Möglichkeit vorstellen, die sowohl für Ihre Mitarbeitenden als auch als Firmen-Investment von Interesse sein könnte.

In der Nähe Ihres Firmensitzes in Hartberg, nur etwa 20 Minuten entfernt, befindet sich unsere letzte verfügbare Einheit im Neubauprojekt „Haus 3 – St. Johann i. d. Haide 247“. Diese 3-Zimmer-Gartenwohnung bietet nicht nur eine großzügige Wohnfläche von 71,64 m² und einen eigenen Garten, sondern ist auch mit modernen Annehmlichkeiten wie einer Fußbodenheizung und einer zentralen Luft-/Wasser-Wärmepumpe ausgestattet. Die Wohnung ist barrierefrei und sofort bezugsfertig, was sie zu einer idealen Wahl für Ihre Mitarbeitenden macht.

Da es sich um die letzte verfügbare Einheit handelt, besteht eine exklusive Gelegenheit, die nicht lange bestehen wi

In [24]:
import os
from utils.utils_llm import query_perplexity, gpt_chat

# Ensure outputs directory exists
os.makedirs("outputs", exist_ok=True)

# Loop through all entries in info_strings and generate research and email for each
for idx, contact_info in enumerate(info_strings):
    # 1. Perplexity research
    research_prompt = f"Bitte mache eine ausführliche Web-Recherche zu folgender Person/Firma und gib alle relevanten Erkenntnisse strukturiert zurück.\n\n{contact_info}. Also find out how close the company is to St. Johann i. d. Haide 247"
    research_notes = query_perplexity(research_prompt)

    # 2. Fill the email prompt
    project_data = product_description if 'product_description' in locals() else ""
    filled_prompt = email_prompt.replace("{{ContactInfo}}", contact_info)
    filled_prompt = filled_prompt.replace("{{ResearchNotes}}", research_notes)
    filled_prompt = filled_prompt.replace("{{ProjectData}}", project_data)

    messages = [
        {"role": "system", "content": "Du bist ein professioneller Immobilienverkäufer."},
        {"role": "user", "content": filled_prompt}
    ]
    email_text = gpt_chat(messages, model="gpt-4o", temperature=0.4, max_tokens=512)

    # 3. Write output to file
    with open(f"outputs/email_{idx+1}.txt", "w") as f:
        f.write("==== RESEARCH ====" + "\n" + research_notes + "\n\n")
        f.write("==== EMAIL ====" + "\n" + email_text + "\n")
    print(f"Generated email for entry {idx+1}")

Generated email for entry 1
Generated email for entry 2
Generated email for entry 2
Generated email for entry 3
Generated email for entry 3
Generated email for entry 4
Generated email for entry 4
Generated email for entry 5
Generated email for entry 5
Generated email for entry 6
Generated email for entry 6
Generated email for entry 7
Generated email for entry 7
Generated email for entry 8
Generated email for entry 8
Generated email for entry 9
Generated email for entry 9
Generated email for entry 10
Generated email for entry 10
Generated email for entry 11
Generated email for entry 11
Generated email for entry 12
Generated email for entry 12
Generated email for entry 13
Generated email for entry 13
Generated email for entry 14
Generated email for entry 14
Generated email for entry 15
Generated email for entry 15
Generated email for entry 16
Generated email for entry 16
Generated email for entry 17
Generated email for entry 17
Generated email for entry 18
Generated email for entry 18
Ge